In [1]:
import os
import sys
import time

import matplotlib.pyplot as plt
from qarpo.demoutils import liveQstat, progressIndicator, summaryPlot, videoHTML
from qarpo.model_visualizer_link import showModelVisualizerLink

print("Imported Python modules successfully.")

Imported Python modules successfully.


In [2]:
# Create FP16 IR files
!mo \
--input_model raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel \
--compress_to_fp16   \
--output_dir models/mobilenet-ssd/FP16 \
--scale 256 \
--mean_values [127,127,127] 

# Create FP32 IR files
!mo \
--input_model raw_models/public/mobilenet-ssd/mobilenet-ssd.caffemodel \
--output_dir models/mobilenet-ssd/FP32 \
--scale 256 \
--mean_values [127,127,127] 

# find all resulting IR files
!echo "\nAll IR files that were created:"
!find ./models -name "*.xml" -o -name "*.bin" 

[ INFO ] Generated IR will be compressed to FP16. If you get lower accuracy, please consider disabling compression explicitly by adding argument --compress_to_fp16=False.
Find more information about compression to FP16 at https://docs.openvino.ai/2023.0/openvino_docs_MO_DG_FP16_Compression.html
[ INFO ] The model was converted to IR v11, the latest model format that corresponds to the source DL framework input/output format. While IR v11 is backwards compatible with OpenVINO Inference Engine API v1.0, please use API v2.0 (as of 2022.1) to take advantage of the latest improvements in IR v11.
Find more information about API v2.0 and IR v11 at https://docs.openvino.ai/2023.0/openvino_2_0_transition_guide.html
[ SUCCESS ] Generated IR version 11 model.
[ SUCCESS ] XML file: /home/u191360/car_object_detection/models/mobilenet-ssd/FP16/mobilenet-ssd.xml
[ SUCCESS ] BIN file: /home/u191360/car_object_detection/models/mobilenet-ssd/FP16/mobilenet-ssd.bin
[ INFO ] Generated IR will be compresse

In [3]:
from qarpo.demoutils import videoHTML

# create link and adjust video path to be able to display from /data using videoHTML()
!ln -sfn /data data
videoHTML(
    "Cars Video", ["./cars_1900.mp4"]
)

In [7]:
%%writefile object_detection_job.sh

# Store input arguments: <output_directory> <device> <fp_precision> <input_file> <num_reqs>
OUTPUT_FILE=$1
DEVICE=$2
FP_MODEL=$3
INPUT_FILE=$4
NUM_REQS=$5

echo VENV_PATH=$VENV_PATH
echo INPUT_FILE=$INPUT_FILE
echo FP_MODEL=$FP_MODEL
echo INPUT_TILE=$INPUT_FILE
echo NUM_REQS=$NUM_REQS

# Follow this order of setting up environment for openVINO 2022.1.0.553
echo "Activating a Python virtual environment from ${VENV_PATH}..."
source ${VENV_PATH}/bin/activate

# The default path for the job is the user's home directory,
#  change directory to where the files are.
cd $PBS_O_WORKDIR

# Make sure that the output directory exists.
mkdir -p $OUTPUT_FILE

# Set inference model IR files using specified precision
MODELPATH=models/mobilenet-ssd/${FP_MODEL}/mobilenet-ssd.xml
LABEL_FILE=./labels.txt

# Run the object detection code
python3 object_detection.py -m $MODELPATH \
                            -i $INPUT_FILE \
                            -o $OUTPUT_FILE \
                            -d $DEVICE \
                            -nireq $NUM_REQS \
                            --labels $LABEL_FILE

# Run the output video annotator code
SCALE_FRAME_RATE=1    # scale number or output frames to input frames
SCALE_RESOLUTION=0.5  # scale output frame resolution 
python3 object_detection_annotate.py -i $INPUT_FILE \
                                     -o $OUTPUT_FILE \
                                     -f $SCALE_FRAME_RATE \
                                     -s $SCALE_RESOLUTION


Overwriting object_detection_job.sh


In [8]:
# import necessary libraries
import json

import qarpo

# load job configurations for demo
with open("./job_config.json") as json_file:
    data = json.load(json_file)

# create and run the user job interface
job_interface = qarpo.Interface(data)
job_interface.displayUI()

Button(button_style='info', description='Submit', style=ButtonStyle())